In [1]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

from keras.preprocessing.image import ImageDataGenerator,array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        width_shift_range=0.2,
        height_shift_range=0.2,     
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')


Using TensorFlow backend.


In [12]:
img = load_img('data/train/Good/18NCGT0014KIN_Beauregard_G2_6002_NIR_V1_O1006.tif')  # this is a sweetpotato image
x = img_to_array(img)  # this is a Numpy array with shape
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='preview', save_prefix='sweet_potato', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers.normalization import BatchNormalization

model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(384,512,1)))
model.add(Activation('relu'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64,(5, 5)))
model.add(Activation('relu'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(512))
model.add(Activation('relu'))
model.add(BatchNormalization(axis=-1))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [3]:
batch_size =10

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        width_shift_range=0.2,
        height_shift_range=0.2,  
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(384, 512),  # all images will be resized to 384x512
        batch_size=batch_size,
        color_mode='grayscale',
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'data/validation',
        target_size=(384, 512),
        batch_size=batch_size,
        color_mode='grayscale',
        class_mode='categorical')

Found 89 images belonging to 2 classes.
Found 89 images belonging to 2 classes.


In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=9,
        epochs=5,
        validation_data=validation_generator,
        validation_steps=50)
model.save_weights('first_try.h5')  # always save your weights after training or during training

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
9/9 [==============================] - 558s 62s/step - loss: 1.7313 - acc: 0.5929 - val_loss: 4.8863 - val_acc: 0.4000
Epoch 2/5
9/9 [==============================] - 604s 67s/step - loss: 1.2776 - acc: 0.5973 - val_loss: 4.0737 - val_acc: 0.4170
Epoch 3/5
5/9 [===============>..............] - ETA: 5:16 - loss: 1.2707 - acc: 0.5200

In [6]:
test_gen = test_datagen.flow_from_directory(
        'data/test',
        target_size=(384, 512),
        batch_size=1,
        color_mode='grayscale',
    class_mode='categorical')
test_gen.reset()  
result=model.predict_generator(test_gen,steps=89)
        

Found 89 images belonging to 1 classes.


In [8]:
import numpy
assigned_class=numpy.argmax(result,axis=1)
import pandas as pd
filenames=pd.DataFrame({'FileName':test_gen.filenames})
new = filenames["FileName"].str.split("\\", n = 1, expand = True) 
filenames["FileName"]=new[1]
filenames.head()

,FileName
0,18NCGT0014KIN_Beauregard_G2_6002_NIR_V1_O1006.tif
1,18NCGT0014KIN_Beauregard_G2_6002_NIR_V1_O1007.tif
2,18NCGT0014KIN_Beauregard_G2_6002_NIR_V1_O1008.tif
3,18NCGT0014KIN_Beauregard_G2_6002_NIR_V1_O1009.tif
4,18NCGT0014KIN_Beauregard_G2_6002_NIR_V1_O1010.tif


In [10]:
mdict=train_generator.class_indices     
classname=numpy.array([*mdict])
print(classname)
filenames["class"]=classname[numpy.argmax(result,axis=1)]
filenames.head(89)

['Curvature' 'Good']


,FileName,class
0,18NCGT0014KIN_Beauregard_G2_6002_NIR_V1_O1006.tif,Curvature
1,18NCGT0014KIN_Beauregard_G2_6002_NIR_V1_O1007.tif,Good
2,18NCGT0014KIN_Beauregard_G2_6002_NIR_V1_O1008.tif,Curvature
3,18NCGT0014KIN_Beauregard_G2_6002_NIR_V1_O1009.tif,Curvature
4,18NCGT0014KIN_Beauregard_G2_6002_NIR_V1_O1010.tif,Curvature
5,18NCGT0014KIN_Beauregard_G2_6002_NIR_V1_O1011.tif,Curvature
6,18NCGT0014KIN_Beauregard_G2_6002_NIR_V1_O1012.tif,Curvature
7,18NCGT0014KIN_Beauregard_G2_6002_NIR_V1_O1013.tif,Curvature
8,18NCGT0014KIN_Beauregard_G2_6002_NIR_V1_O1014.tif,Curvature
9,18NCGT0014KIN_Beauregard_G2_6002_NIR_V1_O1018.tif,Curvature
